In [1]:
!python --version

Python 3.7.11


In [2]:
!pip install -q -U pip
!pip install -q numpy
!pip install -q pandas
!pip install -q ckiptagger
!pip install -q tqdm
!pip install -q tensorflow==1.14.0
!pip install -q ipywidgets
!pip install -q matplotlib

     |████████████████████████████████| 1.6 MB 7.0 MB/s 
     |████████████████████████████████| 109.3 MB 18 kB/s 
     |████████████████████████████████| 50 kB 5.7 MB/s 
     |████████████████████████████████| 488 kB 33.2 MB/s 
     |████████████████████████████████| 3.1 MB 24.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.5 requires tensorflow>=2.0.0, but you have tensorflow 1.14.0 which is incompatible.


In [3]:
import pandas as pd
import numpy as np

from ckiptagger import WS, POS, data_utils
from tqdm.notebook import tqdm
data_utils.download_data_gdown("./")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [4]:
df_train = pd.read_csv('news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv('news_clustering_test.tsv', sep='\t')

In [5]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

test_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
test_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

In [6]:
all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

# 斷詞 + POS

In [7]:
ws = WS('./data/')
pos = POS('./data/')

In [8]:
train_title_cuts = {}
for index, title in tqdm(train_titles.items()):
    word_s = ws([title])
    word_p = pos(word_s)
    train_title_cuts[index] = list(zip(word_s[0], word_p[0]))

  0%|          | 0/1800 [00:00<?, ?it/s]

In [9]:
test_title_cuts = {}
for index, title in tqdm(test_titles.items()):
    word_s = ws([title])
    word_p = pos(word_s)
    test_title_cuts[index] = list(zip(word_s[0], word_p[0]))

  0%|          | 0/600 [00:00<?, ?it/s]

# 尋找降維的詞向量：PPMI + SVD

In [11]:
word2index = {}
index2word = {}
n = 0
for index in train_title_cuts:
    for word, flag in train_title_cuts[index]:
        if word in word2index:
            continue
        word2index[word] = n 
        index2word[n] = word
        n += 1

In [12]:
len(word2index)

6690

如果使用one-hot就需要這麼大的維度的詞向量

In [29]:
# 建立Co-Matrix

vocab_size = len(word2index)
co_matrix = np.zeros(shape=(vocab_size, vocab_size), dtype=np.int32)

window_size = 1
# YOUR CODE HERE
for entries in train_title_cuts.values():
  # asterisk for unpacking an iterable into a function call (conversely, if no asterisk is added then it returns an error)
  word, pos = list(zip(*entries))
  for idx, indvwords in enumerate(word):
    bottomline = idx - window_size
    previous_idx = bottomline if bottomline >= 0 else 0
    previous_word = word[previous_idx:idx]
    word_idx = word2index[indvwords]
    for intervals in previous_word:
      index_num = word2index[intervals]
      co_matrix[word_idx, index_num] += 1
      co_matrix[index_num, word_idx] += 1

# END YOUR CODE

In [ ]:
co_matrix

array([[0, 2, 0, ..., 0, 0, 0],
       [2, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

# PMI calculation - equation

$PMI = log_2\frac{P(x,y}{P(x)P(y)} = log_2\frac{\frac{C(x,y)}{N}}{\frac{C(x)}{N}\frac{C(y)}{N}} = log_2\frac{C(x,y)*N}{C(x)C(y)}$

# PPMI calculation - equation

One problem with PMI is that when it encounters 2 zeros, the program outputs an infinite negative number, so to fix this: 

$ PPMI = max(0, PMI(x,y)) $

In [34]:
# 建立PPMI

def get_ppmi(co_matrix: np.ndarray, eps: float=1e-8):
    # YOUR CODE HERE
    # create an array of zero of the same shape as co_matrix
    M = np.zeros_like(co_matrix, dtype=np.float32)
    # calculate C(x), C(y), C(x,y)
    total_x = np.sum(co_matrix) # sum all values in the array
    total_y = np.sum(co_matrix, axis = 0) # by axis 0 (vertically)
    
    # iterate through all dimensions of the co_matrix
    for i in tqdm(range(co_matrix.shape[0])):
      for j in range(co_matrix.shape[1]):
        pmi = np.log2(co_matrix[i, j] * total_x / (total_y[i] * total_y[j] + eps))
        M[i,j] = max(0, pmi) # fix problem with pmi - ppmi
    # END YOUR CODE
    return M

ppmi = get_ppmi(co_matrix)

  0%|          | 0/6690 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log2
  # This is added back by InteractiveShellApp.init_path()


In [36]:
ppmi

array([[ 0.      ,  9.267157,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 9.267157,  0.      , 11.515084, ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      , 11.515084,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       ...,
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ]], dtype=float32)

In [43]:
# 進行SVD分解，並得到降維的詞向量

from sklearn.decomposition import TruncatedSVD

# 使用`TruncatedSVD`進行降維，降維到dim=1000
# YOUR CODE HERE
svd = TruncatedSVD(n_components = 1000)
word_vectors = svd.fit_transform(ppmi) # use fit_transform() so that a list will be returned and has attribute .shape
# END YOUR CODE

In [44]:
word_vectors.shape

(6690, 1000)

# 新的詞向量 + Group mean vector: 測試

In [45]:
excluded_flags = [
    'Nh', 'Nep', 'Nes', 'DE', 'T', 'P', 'V_2', 'SHI',
    'Dfa', 'Dfb', 'Da', 'Di', 'Dk',
    'Caa', 'Cab', 'Cba', 'Cbb',
    'COLONCATEGORY', 'COMMACATEGORY', 'DASHCATEGORY', 'DOTCATEGORY', 'ETCCATEGORY', 'EXCLAMATIONCATEGORY',
    'PARENTHESISCATEGORY', 'PAUSECATEGORY', 'PERIODCATEGORY', 'QUESTIONCATEGORY', 'SEMICOLONCATEGORY',
    'SPCHANGECATEGORY', 'WHITESPACE'
]

In [46]:
train_svd_vectors = {}
for index, pairs in train_title_cuts.items():
    selected_word_vectors = []
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            selected_word_vectors.append(word_vectors[word2index[word], :])
    vector = np.sum(selected_word_vectors, axis=0)
    if np.sum(np.square(vector)) == 0:
        continue
    train_svd_vectors[index] = vector
    

In [47]:
test_svd_vectors = {}
for index, pairs in test_title_cuts.items():
    selected_word_vectors = []
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            selected_word_vectors.append(word_vectors[word2index[word], :])
    vector = np.sum(selected_word_vectors, axis=0)
    if np.sum(np.square(vector)) == 0:
        continue
    test_svd_vectors[index] = vector

In [48]:
group_vectors = {news_class: [] for news_class in all_news_class}
for index, vector in sorted(train_svd_vectors.items()):
    news_class = train_classes[index]
    group_vectors[news_class].append(vector)

group_mean_vector = {}
for news_class, vectors in group_vectors.items():
    group_mean_vector[news_class] = np.mean(vectors, axis=0)

In [49]:
def cosine_similarity(bow1, bow2):
    len_bow1 = np.sqrt(np.sum(np.square(bow1)))
    len_bow2 = np.sqrt(np.sum(np.square(bow2)))
    return np.sum(bow1 * bow2) / (len_bow1 * len_bow2)

In [50]:
classification = {news_class: [] for news_class in all_news_class}
for index, vector in sorted(test_svd_vectors.items()):
    if np.sum(np.square(vector)) == 0:
        continue

    max_val = -2.0
    max_class = None
    for news_class, ref_vector in group_mean_vector.items():
        val = cosine_similarity(ref_vector, vector)
        if val > max_val:
            max_class = news_class
            max_val = val

    classification[max_class].append(index)

In [51]:
from collections import Counter

for group, ids in classification.items():
    counter = Counter([test_classes[id] for id in ids])
    print('predict', group, ': ', counter)
    total_news = sum(counter[num] for num in counter)
    acc = counter[group]/ total_news
    print("accuracy: {:.2f}".format(acc))

predict 體育 :  Counter({'體育': 62, '旅遊': 9, '遊戲': 9, '財經': 8, '科技': 5, '農業': 4})
accuracy: 0.64
predict 財經 :  Counter({'財經': 62, '科技': 23, '農業': 16, '體育': 8, '遊戲': 8, '旅遊': 7})
accuracy: 0.50
predict 科技 :  Counter({'科技': 52, '體育': 15, '財經': 14, '農業': 9, '遊戲': 9, '旅遊': 8})
accuracy: 0.49
predict 旅遊 :  Counter({'旅遊': 58, '農業': 11, '科技': 5, '財經': 4, '體育': 2, '遊戲': 2})
accuracy: 0.71
predict 農業 :  Counter({'農業': 58, '旅遊': 7, '體育': 4, '財經': 4, '遊戲': 4, '科技': 3})
accuracy: 0.72
predict 遊戲 :  Counter({'遊戲': 68, '科技': 11, '旅遊': 9, '財經': 8, '體育': 7, '農業': 1})
accuracy: 0.65
